# Vector Store using Pinecone

#### create environment variables

GOOGLE_API_KEY="YOUR_GOOGLE_API_KEY"

LANGSMITH_API_KEY="LANGSMITH_API_KEY"

LANGSMITH_TRACING="true"

PINECONE_API_KEY="PINECONE_API_KEY"


### importing necessary libraries

In [47]:
import langchain
from pinecone import Pinecone
from langchain_pinecone import PineconeEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv


In [2]:
load_dotenv()

True

#### Loading the document

In [ ]:
loader = PyPDFLoader("D:\\software\python_vs\\vector_stores\\1706.03762v7 (1).pdf")
data = loader.load()

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\rppaw\AppData\Local\Temp\ipykernel_12340\2893313649.py:1: SyntaxWarning: invalid escape sequence '\p'
  loader = PyPDFLoader("D:\\software\python_vs\\vector_stores\\1706.03762v7 (1).pdf")


In [8]:
print(len(data))
print(f"{data[0].metadata}\n")
print(data[0].page_content)

15
{'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'D:\\software\\python_vs\\vector_stores\\1706.03762v7 (1).pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@g

In [11]:
content = ""
for i in range(len(data)):
    content += data[i].page_content
content

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurren

##### Converting the large text into small chunks

In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len,
    is_separator_regex=False
)

texts = text_splitter.create_documents([content])

In [14]:
texts

[Document(metadata={}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention 

#### Initializing the Embedding Model

In [19]:
embeddings = PineconeEmbeddings(model="multilingual-e5-large")

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000292ED086B10>


In [22]:
len(docs_embeds[0])

1024

In [24]:
query = "how attention function can be described "
query_embed = embeddings.embed_query(query)
query_embed

[0.035614013671875,
 -0.022796630859375,
 -0.037567138671875,
 -0.0309295654296875,
 0.0302734375,
 -0.01471710205078125,
 -0.0233001708984375,
 0.0855712890625,
 0.060302734375,
 0.0080413818359375,
 0.0304412841796875,
 0.01125335693359375,
 -0.045135498046875,
 0.00848388671875,
 -0.0290679931640625,
 -0.00983428955078125,
 -0.04180908203125,
 0.011444091796875,
 -0.0164031982421875,
 0.0032939910888671875,
 0.02618408203125,
 -0.016815185546875,
 -0.053741455078125,
 -0.033966064453125,
 -0.0261993408203125,
 0.01226043701171875,
 -0.0248260498046875,
 -0.02764892578125,
 -0.00933837890625,
 -0.0160675048828125,
 -0.0274200439453125,
 -0.01059722900390625,
 -0.0340576171875,
 -0.0091552734375,
 -0.0240936279296875,
 0.059661865234375,
 0.0242462158203125,
 0.01186370849609375,
 -0.04278564453125,
 0.022918701171875,
 -0.020416259765625,
 0.0306549072265625,
 -0.0117645263671875,
 -0.027923583984375,
 0.0013856887817382812,
 0.04864501953125,
 0.022735595703125,
 -0.0283050537109375

#### setting up the cloud vector store

In [57]:
pc = Pinecone()
index = pc.Index("new")

vector_store = PineconeVectorStore.from_documents(
    documents=texts,
    embedding=embeddings,
    index_name="new"
)

#### getting the documnets from the pinecone

In [58]:
docs_search = PineconeVectorStore.from_existing_index(
    index_name="new",
    embedding=embeddings
)

In [59]:
query = "Advances in Neural Information Processing Systems"

In [60]:
docs_search.similarity_search(query,k=10)

[Document(id='fbfc138b-c5be-4ca4-90aa-1434b7b444be', metadata={}, page_content='the limits of language modeling. arXiv preprint arXiv:1602.02410, 2016.\n[16] Łukasz Kaiser and Samy Bengio. Can active memory replace attention? In Advances in Neural\nInformation Processing Systems, (NIPS), 2016.\n[17] Łukasz Kaiser and Ilya Sutskever. Neural GPUs learn algorithms. In International Conference\non Learning Representations (ICLR), 2016.\n[18] Nal Kalchbrenner, Lasse Espeholt, Karen Simonyan, Aaron van den Oord, Alex Graves, and Ko-\nray Kavukcuoglu. Neural machine translation in linear time.arXiv preprint arXiv:1610.10099v2,\n2017.\n[19] Yoon Kim, Carl Denton, Luong Hoang, and Alexander M. Rush. Structured attention networks.\nIn International Conference on Learning Representations, 2017.\n[20] Diederik Kingma and Jimmy Ba. Adam: A method for stochastic optimization. In ICLR, 2015.\n[21] Oleksii Kuchaiev and Boris Ginsburg. Factorization tricks for LSTM networks. arXiv preprint\narXiv:1703.

## Creating chatbot on google gen ai chatmodel along with pinecone

In [61]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
)

In [62]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docs_search.as_retriever()  
)

In [63]:
qa.invoke(query)

{'query': 'Advances in Neural Information Processing Systems',
 'result': "This venue appears in references [16] and [37].  It's abbreviated as NIPS in [16] and given in full in [37]."}